# File Description

The purpose of this file is to test some visualisations of the data of bathymetry of Tarawa. The data comes from www.pacgeo.org
There are two groups of data: Ocean side and Lagoon side.
    
The details on the webpage and the metadata are pretty vague. Not sure at this stage what the coordinate system is.

The data was download as zipped shapefiles from:
    -http://www.pacgeo.org/layers/geonode:ki_tarawa_ocean_side_mbes_bathy_20m_grid
    -http://www.pacgeo.org/layers/geonode:ki_tarawa_mbes_bathy_lagoon_20m_grid
            
Both are on a 20m grid

In [ ]:
import sys
!{sys.executable} - m pip install wand

In [1]:
# Map of Bathymetry:
from wand.image import Image as WImage
img = WImage(filename='140_ki_tarawa_bathy_1_to_100_000_p59Jd3Q.pdf')
img

ModuleNotFoundError: No module named 'wand'

# File Setup

In [ ]:
import shapefile
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cartopy
import cartopy.crs as ccrs
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
import geopandas as gpd
import sklearn
import scipy.interpolate
import rpy2
import os
os.environ['R_HOME'] = 'C:\ProgramData\Anaconda3\Lib\R'

from matplotlib.ticker import MultipleLocator
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import math

%load_ext rpy2.ipython
# from notebook.services.config import ConfigManager
# cm = ConfigManager().update('notebook', {'limit_output': 10})

# Data Import

In [ ]:
# Load the data using geopandas

folder_loc = r'C:\Users\shannonb\Documents\Model_and_data\Dataset\D1_bathymetry\Model_and_data\Dataset\D1_bathymetry'

gdf_lagoon = gpd.read_file('{}\ki_tarawa_mbes_bathy_lagoon_20m_grid.shp'.format(folder_loc))
gdf_ocean = gpd.read_file('{}\ki_tarawa_ocean_side_mbes_bathy_20m_grid.shp'.format(folder_loc))

# convert the geopandas to pandas
df_lagoon = pd.DataFrame(gdf_lagoon)
df_ocean = pd.DataFrame(gdf_ocean)

# # Sample the data to make it faster....
# df_ocean = df_ocean.sample(1000)
# df_lagoon = df_lagoon.sample(1000)

# Convert the data to float
df_ocean = df_ocean[['X','Y','A']].astype({'X':float,'Y':float,'A':float})
df_lagoon = df_lagoon[['X','Y','A']].astype({'X':float,'Y':float,'A':float})

# Combine the dataframes
df_bath = df_ocean.append(df_lagoon)

In [ ]:
df_bath = df_ocean

In [ ]:
df_test = df_bath.copy()

grid_size = 50

# Group into grid
df_test['X'] = (round(df_test.X/grid_size))*grid_size
df_test['Y'] = (round(df_test.Y/grid_size))*grid_size

# Make depth positive
df_test['A'] = -df_test['A']

# Convert to km
df_test['X'] = df_test['X']/1000.0
df_test['Y'] = df_test['Y']/1000.0

# Chop off part that's not the tarawa atoll
df_test = df_test[df_test.Y<185]

# Make log scale
exp = 0.5
df_test['A'] = [x**exp for x in df_test['A']]

df_pivot = df_test.pivot_table(index='Y',columns='X')['A']
# test_array = np.array(df_test)

In [ ]:
cmap='coolwarm'

font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 15}
matplotlib.rc('font', **font)

fig = plt.figure(figsize=(15,15))

ax = plt.subplot2grid((1,1),(0,0))

X,Y = np.meshgrid(list(df_pivot.columns),list(df_pivot.index))

output = ax.contourf(X,Y,df_pivot,cmap=cmap,levels=100)

ax.set_ylabel('Latitude (km)')
ax.set_xlabel('Longitude (km)')
ax.set_title('Tarawa Atoll Bathymetry')

ax.xaxis.set_minor_locator(MultipleLocator(1))
ax.yaxis.set_minor_locator(MultipleLocator(1))

ax.set_xlim(np.min(ax.get_xlim())-2.5,np.max(ax.get_xlim())+2.5)
ax.set_ylim(np.min(ax.get_ylim())-2.5,np.max(ax.get_ylim())+2.5)

ax.tick_params(axis='x', which='minor', bottom=False)

ax.grid(True,which='both',axis='both')

cax = inset_axes(ax,
                    width="5%",  # width = 50% of parent_bbox width
                    height="50%",  # height : 5%
                    loc='upper right',
                    borderpad=4)

cbar = fig.colorbar(output,cax=cax)
cax.set_title('Depth (m)')
y_ticks =  cax.get_yticks()
# cbar.set_ticklabels([round(y**(1/exp)) for y in y_ticks])

levels = [1500,1250,1000,750,500,400,300,200,100,50,25,10,5]
levels_exp = [x**(exp) for x in levels]
cbar.set_ticks(levels_exp)
cbar.set_ticklabels(levels)

plt.show()


# Making Some transects

## Data for checking the transects

In [ ]:


df_pivot_filtered = df_pivot.loc[:,(df_pivot.columns<717.5)&(df_pivot.columns>716.5)]
df_pivot_filtered = df_pivot_filtered[(df_pivot_filtered.index<146.5)&(df_pivot_filtered.index>143.5)]
df_pivot_filtered


## Test Point

In [ ]:
import statsmodels.api as sm
from scipy import interpolate
from numpy import diff
# from kneed import KneeLocator
# x = np.arange(741,744,(744-741)/100.0)
# y = np.arange(153,154,(148-147)/100.0)#[147]*len(x)

def moving_average(a, n) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return(ret[n - 1:] / n)

def transect_determination(df_bathymetry,n):
    '''
    inputs:
        - n is period of moving average
    '''
    
    # Set location for x and y transect temporarily
#     x = [721]*50
#     y = np.arange(146,143,(143-146)/50)
    x = np.arange(712,708,(712-708)/100.0)
    y = np.arange(153,154,(154-152)/100.0) # [147]*len(x)
#     x = np.arange(718,717.0,-(718-717)/100.0)
#     y = np.arange(147,143.0,-(147-143)/100.0)

    # Get the coordinates and depths of the island
    points = np.array(df_bathymetry[['X','Y']])
    values = np.array(df_bathymetry['A'])

    # Calculate the depth along the transect coordinates x and y
    transect_depth = interpolate.griddata(points,values,(x,y),method='cubic')
    
    # Create a transect dataframe with coordinates and depths
    df_transect = pd.DataFrame({'long':x,'lat':y,'depth':transect_depth})
    df_transect.dropna(inplace=True)
    
    # Set latitude and longtidue to be the first value in the transect #### REPLACE WITH SHORE VALUE WHEN I HAVE IT
    zero_point_long = df_transect.loc[0].long
    zero_point_lat = df_transect.loc[0].lat
    
    # Calculate distance from zero point (future: shore)
    df_transect['distance'] = np.sqrt((df_transect.long-zero_point_long)**2+\
                                        (df_transect.lat-zero_point_lat)**2)
    
    # Make sure transect is in order of distance
    df_transect.sort_values('distance',inplace=True)
    
    # Put a moving average through the depth
    depth_moving_avg = list(moving_average(list(df_transect.depth),n))
    length_of_nan = [np.nan]*int((n-1)/2)

    # Add to df_transect
    df_transect['depth_moving_avg'] = length_of_nan+depth_moving_avg+length_of_nan
    df_transect.dropna(inplace=True,axis=0)
    
    return(df_transect)
    
def determine_crest_location(df_transect):
    '''
    '''
    
    # Find the second derivative
    depth_dash = diff(df_transect.depth_moving_avg)/diff(df_transect.distance)
    distance_average = [np.mean([x1,x2]) for x1,x2 in zip(df_transect.distance[1:],df_transect.distance[:-1])]
    
    df_dash = pd.DataFrame({
                'distance_average':distance_average,
                'depth_dash':depth_dash
            })
    
    df_dash.dropna(inplace=True,axis=0)
    
    # Find the double derivative
    depth_dashdash = np.diff(df_dash.depth_dash)/np.diff(df_dash.distance_average)
    distance_average_average = [np.mean([x1,x2]) for x1,x2 in zip(df_dash.distance_average[1:],df_dash.distance_average[:-1])]
    df_dashdash = pd.DataFrame({
                'distance_average_average':[abs(x) for x in distance_average_average],
                'depth_dashdash':depth_dashdash
            })
        
    # The elbow should be at the maximum in the double derivative
    df_crest_loc_deriv = df_dashdash[df_dashdash.depth_dashdash==np.max(df_dashdash.depth_dashdash)]
    
    # Get the distance from shore
    crest_dist_shore = df_crest_loc_deriv.reset_index().loc[0,'distance_average_average']
    
    # Get the location of the crest not in the derivative df    
    df_crest_loc = df_transect[\
               abs(df_transect.distance-crest_dist_shore)==np.min(abs(df_transect.distance-crest_dist_shore))]
    
    return(df_dashdash,df_crest_loc)

def beta_calculation(df_transect,df_crest_loc):
    '''
    
    '''
    
    # Divide the transect into reef flat and fore reef
    df_fore_reef = df_transect.loc[[x for x in df_transect.index if x>df_crest_loc.index[0]]]
    df_reef_flat = df_transect.loc[[x for x in df_transect.index if x<df_crest_loc.index[0]]]

    # Fit a linear regression of reef flat
    reef_flat_distance = sm.add_constant(list(df_reef_flat.distance)) # create a var for this algorithm for distance
    lin_reg_reef_flat = sm.OLS(list(df_reef_flat.depth),reef_flat_distance).fit()
    lin_reg_reef_flat_pred = lin_reg_reef_flat.predict(reef_flat_distance)
    
    # Fit linear regression to fore reef
    fore_reef_distance = sm.add_constant(list(df_fore_reef.distance)) # create a var for this algorithm for distance
    lin_reg_fore_reef = sm.OLS(list(df_fore_reef.depth),fore_reef_distance).fit()
    lin_reg_fore_reef_pred = lin_reg_fore_reef.predict(fore_reef_distance)
    
    # Create dataframes of linear regression results
    df_reef_flat_regression= pd.DataFrame({'distance':list(df_reef_flat.distance),'regression':lin_reg_reef_flat_pred})
    df_fore_reef_regression = pd.DataFrame({'distance':list(df_fore_reef.distance),'regression':lin_reg_fore_reef_pred})
    
    # Calculate beta using linear regression slopes
    m_reef_flat = lin_reg_reef_flat.params[1]
    m_fore_reef = lin_reg_fore_reef.params[1]
    tan_beta = (m_fore_reef-m_reef_flat)/(1+m_fore_reef*m_reef_flat)
    beta = math.atan(tan_beta)
    
    return(df_reef_flat_regression,df_fore_reef_regression,beta)

In [ ]:
fig = plt.figure(figsize=(10,10))

ax1 = plt.subplot2grid((2,1),(0,0))
ax2 = plt.subplot2grid((2,1),(1,0))

df_transect = transect_determination(df_test,21)

ax1.plot(df_transect.distance,df_transect.depth_moving_avg)
ax1.set_ylabel('Depth (m)')
ax1.set_xlabel('Distance (km)')
ax1.invert_yaxis()

df_dashdash,df_crest_loc = determine_crest_location(df_transect)

ax2.plot(df_dashdash.distance_average_average,df_dashdash.depth_dashdash)
ax1.scatter(df_crest_loc.distance,df_crest_loc.depth_moving_avg)

df_reef_flat_regression,df_fore_reef_regression,beta = beta_calculation(df_transect,df_crest_loc)

ax1.plot(df_reef_flat_regression.distance,df_reef_flat_regression.regression)
ax1.plot(df_fore_reef_regression.distance,df_fore_reef_regression.regression)

plt.show()


##############################################

## Get Island Core

In [ ]:
df_island_and_lagoon = df_pivot.copy()

df_island_and_lagoon[~df_island_and_lagoon.isna()] = 0
df_island_and_lagoon[df_island_and_lagoon.isna()] = 1
df_island_and_lagoon[df_island_and_lagoon==0] = np.nan



In [ ]:
cmap='coolwarm'

font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 15}
matplotlib.rc('font', **font)

fig = plt.figure(figsize=(15,15))

ax = plt.subplot2grid((1,1),(0,0))

X,Y = np.meshgrid(list(df_island_and_lagoon.columns),list(df_island_and_lagoon.index))

output = ax.contourf(X,Y,df_island_and_lagoon,cmap=cmap,levels=100)

ax.set_ylabel('Latitude (km)')
ax.set_xlabel('Longitude (km)')
ax.set_title('Tarawa Atoll Bathymetry')

ax.xaxis.set_minor_locator(MultipleLocator(1))
ax.yaxis.set_minor_locator(MultipleLocator(1))

ax.set_xlim(np.min(ax.get_xlim())-2.5,np.max(ax.get_xlim())+2.5)
ax.set_ylim(np.min(ax.get_ylim())-2.5,np.max(ax.get_ylim())+2.5)

ax.tick_params(axis='x', which='minor', bottom=False)

ax.grid(True,which='both',axis='both')

cax = inset_axes(ax,
                    width="5%",  # width = 50% of parent_bbox width
                    height="50%",  # height : 5%
                    loc='upper right',
                    borderpad=4)

cbar = fig.colorbar(output,cax=cax)
cax.set_title('Depth (m)')
y_ticks =  cax.get_yticks()
# cbar.set_ticklabels([round(y**(1/exp)) for y in y_ticks])

levels = [1500,1250,1000,750,500,400,300,200,100,50,25,10,5]
levels_exp = [x**(exp) for x in levels]
cbar.set_ticks(levels_exp)
cbar.set_ticklabels(levels)

plt.show()


# Just trying some grouping...

In [ ]:
plt.scatter(df_test.X,df_test.Y,c=df_test.A)

In [ ]:
%%R -i df_test

plot(x=df_test$X,y=df_test$Y, pch=20)

##  Now get the polygons
ConvexHull = chull(df_test$X, df_test$Y)
polygon(df_test[ConvexHull,], 
    border=rainbow(4)[1], col=rainbow(4, alpha=0.1)[1])

In [ ]:
points = np.array(df_test[['X','Y']])

hull = scipy.spatial.ConvexHull(points)

for simplex in hull.simplices:
    plt.plot(points[simplex, 0], points[simplex, 1], 'k-')


In [ ]:
np.array(df_test[['X','Y']])

In [ ]:
np.random.rand(30, 2)

In [ ]:
from math import sin, cos, sqrt, atan2, radians

lon1 = 72000
lat1 = 16000

for lon2,lat2 in zip(df_test.X,df_test.Y):

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    
    print("Result:", distance)